### Домашнее задание
Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/gb_flask_hw_api)

По шагам:
1. выбрать себе датасет (который интересен или нравится больше всего), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. установить удобную для себя среду разработки (pycharm прекрасен - https://www.jetbrains.com/pycharm/)
3. научиться создавать там виртуальное окружение для python (в правом нижнем углу есть add interpreter)
4. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта конечно же при этом)
5. завести себе аккаунт на guthub (если его еще нет). У самого github есть такой "hello world" по работе с ним - https://guides.github.com/activities/hello-world/
6. научиться запускать gunicorn (установить его конечно). Пример для проекта выше: gunicorn -w 5 -b 127.0.0.1:5000 run_server:app
7. итоговый проект должен содержать: 1) каталог models/ (здесь модель-пайплайн предобученная) 2) файл run_server.py (здесь основной код flask-приложения) 3) requirements.txt (список пакетов, которые у вас используются в проекте) 4) README.md (здесь какое-то описание, что вы делаете, что за данные и т.д)
8. (<b>Опционально</b>): front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

### Задача тематического моделирования заявок в чатбот

In [1]:
import numpy as np
import pandas as pd

from nltk.corpus import names
import nltk; nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords

import re

from pymorphy2 import MorphAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


n_features = 1000
n_components = 16
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sych_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
import dill

In [3]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [4]:
data = pd.read_csv("Заявки в чатбот.csv", encoding = 'utf-16', sep=";")
data.tail(20)

,mesTExt
20136,"Добрый день! В базе Д11 по арт 449499 Свинина охл вес в карточке товара цена SSP и цена продажи не совпадают, ценник..."
20137,"Добрый день. Не работает апс Выключаешь его, включаешь и при включении компьютера он начинает пищать Подключился на ..."
20138,"Добрый день.В автозаказе поставщик Новозеланские продукты не открыта карточка товара код 168181 киви Голд 1шт,Новая ..."
20139,Добрый день Не смогу сменить пароль nan но я ввожу все вернол но я ввожу все вернол ой позвоните еще раз спасибо
20140,Доброго утра: Поставщик Ист Лоджистикал. Подтверждение order все на сегодня nan Спасибо Не осталось
20141,"Добрый день Сегодня была смена юр.лица на самбери 24,при оформление РН,выдал такую ошибку и все документы пустые. na..."
20142,"Доброе утро. \nПрошу перезагрузить RC-LOG59. Спасибо Получилось, спасибо"
20143,"Здравствуйте! ПОдскажите, пожалуйста, как перевести поставщика к EDI? хочет работать так я могу ему направить ДС о п..."
20144,"Да Добрый день.\nНе могу свести заказ, не которые вз не подтягиваются в формирование сводного заказа, например с 01,..."
20145,"Добрый день! Прошу помочь в настройке работы почты на удаленном доступе. Утром подключилась, как обычно, открылись п..."


In [5]:
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
        
    def get_stopwords(self):
        russian_stopwords = stopwords.words("russian")
        df_sw = pd.read_csv('stopwords.csv', encoding = 'utf-8', sep=";")
        for index, row in df_sw.iterrows():
            russian_stopwords.append(row['stopword'])
        return russian_stopwords
        
    def to_lemmatize2(self, df, key):
        all_word_str = " ".join(df[key])
        all_word_list = all_word_str.split()
        all_unique_word = pd.Series(all_word_list).unique()
        lemmatized_word_dict = {}
        lemmatizer = MorphAnalyzer()
        for word in all_unique_word:
            lemmatized_word_dict[word] = lemmatizer.normal_forms(word)[0]
        lemm_func = lambda text: ' '.join([lemmatized_word_dict[word] for word in text.split()])
        df[key] = df[key].apply(lemm_func)
        return df, all_unique_word
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):

        X[self.key] = X[self.key].replace('—','-')
        
        #1. удаляем пунктуацию
        deleted_symbols = r'[\\\\\'[\]!"$%&()*+,-./:;<=>?№@^_`{|}~«»\n]'  
        func = lambda text : re.sub(deleted_symbols, ' ', str(text))
        X[self.key] = X[self.key].apply(func)
        
        #2. удалим смайлики
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        func = lambda text : re.sub(emoji_pattern, ' ', str(text))
        X[self.key] = X[self.key].apply(func)
        
        #3. удалим отдельно стоящие цифры
        func = lambda text : ' '.join([elem for elem in str(text).split(' ') if elem.isdigit() == False])   
        X[self.key] = X[self.key].apply(func)
        
        #4. приводим к нижнему регистру
        X[self.key] = X[self.key].apply(lambda text : text.lower())
        
        #5. лемматизация (приводим слова к начальной форме)
        X, _ = self.to_lemmatize2(X, self.key)
        
        #6. удаляем стоп слова
        sw = self.get_stopwords()
        func = lambda text : ' '.join([elem for elem in str(text).split(' ') if elem not in sw and not elem in ['nan', np.nan]])   
        X[self.key] = X[self.key].apply(func)
        
        return X 
    
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #приведем к виду списка списков, потом этот список списков пойдет в модель LDA
        lst = X[self.key].to_list()
        texts = []
        for i in range(len(lst)):    
            texts.append(lst[i].split(' '))
        return texts

In [6]:
#example
description = Pipeline([
                ('imputer', TextImputer('mesTExt', '')),
                ('selector', ColumnSelector(key='mesTExt'))
            ])

#description.fit(data)
#description.transform(data.iloc[:10])

In [7]:
pipeline = Pipeline([
    ('description', description),
    ('tfidf_vectorizer', TfidfVectorizer(max_df=0.95, min_df=2, analyzer=lambda x: x, 
                                   max_features=n_features,
                                   stop_words='english')),
    ('lda', LatentDirichletAllocation(n_components=n_components, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)),
])

model = pipeline.fit(data.iloc[:])

In [8]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'description', 'tfidf_vectorizer', 'lda', 'description__memory', 'description__steps', 'description__verbose', 'description__imputer', 'description__selector', 'description__imputer__key', 'description__imputer__value', 'description__selector__key', 'tfidf_vectorizer__analyzer', 'tfidf_vectorizer__binary', 'tfidf_vectorizer__decode_error', 'tfidf_vectorizer__dtype', 'tfidf_vectorizer__encoding', 'tfidf_vectorizer__input', 'tfidf_vectorizer__lowercase', 'tfidf_vectorizer__max_df', 'tfidf_vectorizer__max_features', 'tfidf_vectorizer__min_df', 'tfidf_vectorizer__ngram_range', 'tfidf_vectorizer__norm', 'tfidf_vectorizer__preprocessor', 'tfidf_vectorizer__smooth_idf', 'tfidf_vectorizer__stop_words', 'tfidf_vectorizer__strip_accents', 'tfidf_vectorizer__sublinear_tf', 'tfidf_vectorizer__token_pattern', 'tfidf_vectorizer__tokenizer', 'tfidf_vectorizer__use_idf', 'tfidf_vectorizer__vocabulary', 'lda__batch_size', 'lda__doc_topic_prior', 'lda__evaluate_e

In [15]:
data.iloc[:10]

,mesTExt
0,
1,сломаться ноготь сломать ноготь сломать ноготь сломаться ноутбук весь пропасть делать вообще чушь помочь срочно вооб...
2,хабаровск сломаться ноутбук рабртаять автозаказ автозаказ
3,сок протечь сломать ноутбук ерп висеть ерп висеть ерп висеть оператор добавить номер телефон личный кабинет парль
4,
5,ерп ерп ерп упасть напрочь
6,ерп ерп ерп чинить сволочь зависнуть ерп загружаться заказ проводиться это пользователь каждый делать
7,висеть ерп висеть ерп весить erp висеть програм ерп загружаться заказ заказ проводиться документ еклмный проводиться...
8,заказ загружаться проводиться ерп проводиться ерп
9,загружаться заказ висеть ерп проверить весь проводиться туда прилетать заявка странно проводиться проводиться докуме...


In [9]:
#сразу pandas dataframe сделаем
test_preds = pd.DataFrame(pipeline.transform(data.iloc[:10]))
test_preds

C:\Users\sych_\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sych_\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sych_\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.531250
1,0.016578,0.016578,0.016578,0.016578,0.016578,0.016578,0.016578,0.016578,0.016578,0.016578,0.016578,0.016578,0.574099,0.193809,0.016578,0.016578
2,0.181579,0.023388,0.023388,0.023389,0.023388,0.023388,0.023388,0.023388,0.023388,0.023388,0.023388,0.023388,0.023388,0.490984,0.023388,0.023388
3,0.016951,0.016951,0.297728,0.016951,0.016951,0.016951,0.016951,0.016951,0.175048,0.016951,0.243570,0.016951,0.016951,0.016951,0.080246,0.016951
4,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.531250
5,0.031250,0.031250,0.531250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250
6,0.018139,0.018139,0.451061,0.018139,0.018139,0.018139,0.018139,0.018139,0.018139,0.018139,0.018139,0.018139,0.294996,0.018139,0.018139,0.018139
7,0.017122,0.017122,0.632177,0.017122,0.017122,0.017122,0.017122,0.017122,0.017122,0.068247,0.017122,0.017122,0.076996,0.017122,0.017122,0.017122
8,0.021928,0.021928,0.540696,0.021928,0.021928,0.021928,0.021928,0.021928,0.021928,0.152307,0.021928,0.021928,0.021928,0.021928,0.021928,0.021928
9,0.016658,0.016658,0.484221,0.016658,0.016658,0.016658,0.016658,0.016658,0.016658,0.016659,0.016658,0.016658,0.226462,0.016658,0.016658,0.072758


In [18]:
type(pipeline.transform(data.iloc[:10]))

C:\Users\sych_\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sych_\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sych_\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

numpy.ndarray

In [16]:
test_preds.to_csv("test_preds.csv")

Посмотреть на топ слова

In [10]:
print("\nTopics in LDA model:")
tf_feature_names = pipeline.steps[1][1].get_feature_names()
print_top_words(pipeline.steps[2][1], tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: ошибка пко касса весь эник светлый чек pdf камчатка выдавать смена маркетереть дв хабаровск ют печать красный биробиджан печатать смс
Topic #1: документ провести заказ егаиса перемещение вмс ттн весь просить склад возврат вопрос выгрузить рц суворов принтер статус это товар поставщик
Topic #2: ерп зайти программа проводиться сэд выкидывать висеть свет галина карточка лкп весь находка рейс заработать прогрузить база открыть ошибка тсд
Topic #3: заказ поставщик товар цена весь самбрать позиция принять просить код это ошибка заявка 1с автозаказ группа вопрос гм — приёмка
Topic #4: сервер база dev erp ref srvr it rn сетевой тестовый запускаться test or zup обновить доступный конфигуратор хранилище сеанс prog
Topic #5: заказ хороший выгрузиться посмотреть база 1с выгружаться wms подвисать анна прогружаться сильно фин весь виснуть загрузиться висеть стать ух журнал
Topic #6: заявка вопрос закрывать сервис задача просить помочь решить принять большой оплатить к

In [11]:
data.iloc[1]

mesTExt    сломаться ноготь сломать ноготь сломать ноготь сломаться ноутбук весь пропасть делать вообще чушь помочь срочно вооб...
Name: 1, dtype: object

In [12]:
data.head()

,mesTExt
0,
1,сломаться ноготь сломать ноготь сломать ноготь сломаться ноутбук весь пропасть делать вообще чушь помочь срочно вооб...
2,хабаровск сломаться ноутбук рабртаять автозаказ автозаказ
3,сок протечь сломать ноутбук ерп висеть ерп висеть ерп висеть оператор добавить номер телефон личный кабинет парль
4,


In [13]:
len(tf_feature_names)

1000

In [14]:
with open("lda_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)